In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.5 MB/s 
     |████████████████████████████████| 5.8 MB 18.6 MB/s 
     |████████████████████████████████| 1.3 MB 68.1 MB/s 
     |████████████████████████████████| 182 kB 59.8 MB/s 
     |████████████████████████████████| 7.6 MB 51.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c87fb7e5fe21c8fe3c5f833d94b4d5d2012396bb40e50d42f72e8420b07530fe
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

directory = '/content/drive/MyDrive/text/'
metadata = pd.read_json(directory + 'metadata.json', lines=True)

In [ ]:
metadata['titles'] = metadata.apply(lambda x: x['title'] + '. ' + x['sub_title'], axis=1)

# KoBERT

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-l89bnrp0/kobert-tokenizer_e2271e7ee018456da46578d25b4b2636
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-l89bnrp0/kobert-tokenizer_e2271e7ee018456da46578d25b4b2636
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4648 sha256=923dd6a8ff601aadb85b5eb0591e1cb16836ba6d4ea13d8eda14ba5c60ba016e
  Stored in directory: /tmp/pip-ephem-wheel-cache-fwi_qrke/wheels/f7/cb/29/1a737fe71e5108dc30b04ea4a990f78ed271fa537aaf3fce7c
Successfully built kobert-tokenizer


In [ ]:
def encoding(model, tokenizer, sentence):
   inputs = tokenizer.batch_encode_plus([sentence])
   out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
   return out
  # out.pooler_output.shape

In [ ]:
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer

'''
KoBERT
https://github.com/SKTBrain/KoBERT
'''

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
import pickle

with open(directory + 'doc_embedding_KoBERT.pickle', 'rb') as handle:
  doc_embedding = pickle.load(handle)

In [ ]:
len(doc_embedding.keys())

4550

In [ ]:
from tqdm import tqdm
import pickle
import os.path

count = 0
if os.path.exists(directory + 'doc_embedding_KoBERT.pickle'):
    with open(directory + 'doc_embedding_KoBERT.pickle', 'rb') as handle:
      doc_embedding = pickle.load(handle)
else:
  doc_embedding = {}
for id, sentence in tqdm(zip(metadata['id'], metadata['titles'])):
  if id not in doc_embedding.keys():
    doc_embedding[id] = encoding(model, tokenizer, sentence)
    count += 1
  if count % 100 == 0:
    with open(directory + 'doc_embedding_KoBERT.pickle', 'wb') as handle:
      pickle.dump(doc_embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

2584it [1:48:14,  2.43s/it]

In [ ]:
import pickle

directory = '/content/drive/MyDrive/text/'
with open(directory + 'doc_embedding_KoBERT.pickle', 'wb') as handle:
    pickle.dump(doc_embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from tqdm import tqdm

doc_embedding = {}
for id, sentence in tqdm(zip(metadata['id'], metadata['titles'])):
  doc_embedding['id'] = encoding(model, tokenizer, sentence)

643104it [6:39:17, 26.84it/s]


# GPT

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

'''
https://github.com/SKT-AI/KoGPT2
'''

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
text = metadata['titles'][0]

In [ ]:
_ = model.eval()

prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)

In [ ]:
tokens

tensor([[ 9644,  9582, 50788, 10446,  9033, 39105,  7166, 12141,  9430, 30048,
         44515, 10307,  8263, 26478]], device='cuda:0')

In [ ]:
from tqdm import tqdm

doc_embedding = {}
for id, sentence in tqdm(zip(metadata['id'], metadata['titles'])):
  doc_embedding['id'] = encoding(model, tokenizer, sentence)

0it [00:00, ?it/s]


NameError: ignored

In [ ]:
out = encoding(model, tokenizer, sentences[0])

In [ ]:
out['logits'].shape

torch.Size([1, 21, 51200])

In [ ]:
out.keys()

odict_keys(['logits', 'past_key_values'])

In [ ]:
model.transformer.wte.weight.shape

torch.Size([51200, 768])

In [ ]:
tokenizer.batch_encode_plus([sentences[0]])

{'input_ids': [[12224, 9021, 10246, 17947, 9325, 9050, 6969, 8094, 387, 42642, 27251, 43531, 389, 19526, 13400, 9325, 9712, 27531, 9791, 13513, 9292]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

- 유사한 article 찾아보기 similarity로

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 5.5 MB 18.7 MB/s 
     |████████████████████████████████| 1.3 MB 51.1 MB/s 
     |████████████████████████████████| 182 kB 44.9 MB/s 
     |████████████████████████████████| 7.6 MB 42.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c2a8485bf344c25ab2a7a44906979d08283e8bb7f28a6b111714a8a65a6d472e
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image

#Load CLIP model
model = SentenceTransformer('clip-ViT-B-32')

#Encode an image:
img_emb = model.encode(Image.open('동물의숲.jfif'))

#Encode text descriptions
text_emb = model.encode(['There is a tree.', 'A cat on a table', 'A picture of London at night'])

#Compute cosine similarities 
cos_scores = util.cos_sim(img_emb, text_emb)
print(cos_scores)

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


tensor([[0.2496, 0.1704, 0.1882]])


# KoBART

In [ ]:
'''https://github.com/SKT-AI/KoBART'''